# File Name: simple_rag_building_langchain.ipynb
### Location: Chapter 6
### Purpose: 
#####       1. Create an open-source Chroma vector store.  
#####       2. Ingest data into the Vector DB.  
#####       3. Retrieve data with langchain framework 
##### Dependency: Not Applicable
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.34.84" \
    "langchain>=0.2.16" \
    "langchain_community>=0.2.17" \
    "awscli>=1.32.84" \
    "botocore>=1.34.84" \
    "PyPDF2" \
    "pypdf" \
    "langchain-chroma>=0.1.2" \
    "langchain-aws>=0.1.7"   

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import boto3
import botocore
import warnings
import time
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_aws.embeddings.bedrock import BedrockEmbeddings
from langchain_aws import ChatBedrock
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

# Find out data directory

#### 1. Retrieves the current working directory and prints it.
##### 2. Builds a path that navigates up one directory and appends 'data/rag_use_cases' to the path, then prints this resulting path.

In [ ]:
try:
    # Get the current working directory
    current_directory = os.getcwd()
    
    # Print the current working directory
    print(f"Current working directory: {current_directory}")
    
    # Attempt to navigate up one directory and then to 'data/rag_use_cases'
    data_directory = os.path.join(os.path.dirname(current_directory), 'data/rag_use_cases')
    
    # Print the resulting path
    print(f"Data directory path: {data_directory}")
    
except FileNotFoundError as e:
    # Handle the case where the directory path does not exist
    print(f"Error: The specified path does not exist - {e}")
    
except Exception as e:
    # General exception handler for any other errors
    print(f"An unexpected error occurred: {e}")

# Disclaimer
##### Make Sure that data_directory is pointing to the right path and data files are present. Otherwise, you need to change the above code

# Prepare dataset 

##### 1.Parameter Inputs: Accepts data_directory (path to the PDF folder) and documents (list to store PDF data).
##### 2.File Loading Loop: Iterates through all files in the directory, checking if each one has a .pdf extension.
##### 3.PDF Loading with Error Handling: For each PDF, it uses PyPDFLoader to load the content, appending it to documents. If an error occurs, it prints an error message for that specific file, allowing the process to continue.
##### 4.Document Check and Output: If any documents are loaded, it prints the content of the first page from the first document; otherwise, it notifies that no PDFs were found.
##### 5.Function Return: Returns the updated documents list with loaded PDF content.

In [ ]:
def load_pdf_documents(data_directory, documents):
    """
    Load PDF documents from the specified directory and append their content to the documents list.

    Parameters:
    - data_directory (str): The directory path containing the PDF files.
    - documents (list): A list to store the loaded PDF data.

    Returns:
    - list: The updated documents list with the loaded PDF content.
    """
    # Loop through all files in the specified directory
    for filename in os.listdir(data_directory):
        if filename.endswith('.pdf'):  # Check if the file is a PDF
            file_path = os.path.join(data_directory, filename)
            try:
                # Initialize the PDF loader for the current file
                loader = PyPDFLoader(file_path)
                
                # Load the PDF data
                data = loader.load()
                
                # Extend the documents list with the loaded data
                documents.extend(data)
            except Exception as e:
                print(f"Error loading {filename}: {e}")  # Handle exceptions during loading

    # Display the content of the first page of the first document if available
    if documents:
        print(documents[0].page_content)  # Printing page content of the first document
    else:
        print("No PDF files found in the folder.")

    return documents

# Usage
documents = []
documents = load_pdf_documents(data_directory, documents)

# Define prompt, Amazon Bedrock Foundation model, and Amazon Bedrock embed model

In [ ]:
# Define prompt
prompt = "What is Amazon doing and cashflow?"

# List of Bedrock models with names and model codes
bedrock_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# List of Bedrock embed models with names and model codes
bedrock_embed_model_id = "amazon.titan-embed-text-v1"

## Define important environment variable

In [ ]:
# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    # This session is responsible for managing credentials and region configuration
    boto3_session = boto3.session.Session()

    # Retrieve the current AWS region from the session (e.g., 'us-east-1', 'us-west-2')
    aws_region_name = boto3_session.region_name
    
    # Initialize Bedrock and Bedrock Runtime clients using Boto3
    # These clients will allow interactions with Bedrock-related AWS services
    boto3_bedrock_client = boto3.client('bedrock', region_name=aws_region_name)
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=aws_region_name)

    # Store all relevant variables in a dictionary for easier access and management
    variables_store = {
        "aws_region_name": aws_region_name,                          # AWS region name
        "boto3_bedrock_client": boto3_bedrock_client,                # Bedrock client instance
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client,  # Bedrock Runtime client instance
        "boto3_session": boto3_session                               # Current Boto3 session object
    }

    # Print all stored variables for debugging and verification
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

# Handle any exceptions that occur during the execution
except Exception as e:
    # Print the error message if an unexpected error occurs
    print(f"An unexpected error occurred: {e}")

# Section 1

## Create Vector Store with Chroma and ingest data

#### 1. Document Splitting:
###### split_documents function divides large documents into smaller chunks.
###### Uses RecursiveCharacterTextSplitter with specified chunk size and overlap.
###### Returns the splits or logs an error if splitting fails.

#### 2. Vector Store Creation:
###### create_vectorstore function initializes an embedding model (BedrockEmbeddings) and creates a vector store (Chroma).
###### Stores vector representations of document chunks.
###### Returns the vector store or logs an error if it fails.

#### 3. Main Execution Function:
###### create_ingest_vector_store orchestrates the workflow.
###### It first calls split_documents and, upon success, moves to create_vectorstore.
###### Raises an exception if any step fails, logging the issue.

In [ ]:
# Function to split the documents into chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    try:
        # Initialize a recursive character text splitter
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        # Split documents into smaller chunks
        splits = text_splitter.split_documents(documents)
        print(f"Number of splits: {len(splits)}")
        return splits
    except Exception as e:
        # Handle document splitting errors
        print(f"Error while splitting documents: {e}")
        return None

# Function to create embeddings and store vectors using Chroma
def create_vectorstore(splits, bedrock_client, model_id=bedrock_embed_model_id):
    try:
        # Initialize the Bedrock Embeddings model using the Bedrock client
        embeddings_model = BedrockEmbeddings(client=bedrock_client, model_id=model_id)
        
        # Create a vector store with the splits and embeddings model
        vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)
        print("Vector store created successfully.")
        return vectorstore
    except Exception as e:
        # Handle errors during the vector store creation
        print(f"Error while creating vector store: {e}")
        return None

# Main execution function to modularize the flow
def create_ingest_vector_store(documents):
    try:

        # Step 1: Split documents into chunks
        splits = split_documents(documents)
        if not splits:
            raise Exception("Document splitting failed")

        # Step 2: Create a vector store using embeddings
        vectorstore = create_vectorstore(splits, boto3_bedrock_runtime_client)
        if not vectorstore:
            raise Exception("Vector store creation failed")

        # If all steps succeed
        print("Process completed successfully.")
        return vectorstore
        
    except Exception as e:
        # Handle any general errors in the flow
        print(f"An error occurred: {e}")

# Example usage
vectorstore = create_ingest_vector_store(documents)

# Test the Vector DB

#### 1. Document Retrieval Function:
###### retrieve_documents(query, retriever) function takes a query and a retriever to find relevant documents.
###### It attempts to retrieve documents and prints the number retrieved.
###### If any documents are retrieved, it prints the content of the first one; otherwise, it displays a message indicating no documents were found.
###### Errors are caught and printed if retrieval fails.

In [ ]:
# Function to retrieve documents based on a query
def retrieve_documents(query, retriever):
    try:
        # Attempt to retrieve documents based on the query
        retrieved_docs = retriever.invoke(query)
        
        # Print the number of retrieved documents
        print(f"Number of retrieved documents: {len(retrieved_docs)}")
        
        # Print the content of the first retrieved document (if any)
        if len(retrieved_docs) > 0:
            print(retrieved_docs[0].page_content)
        else:
            print("No documents retrieved.")
        
    except Exception as e:
        # Handle any exceptions that arise during retrieval
        print(f"An error occurred while retrieving documents: {e}")

# Example usage of the function
try:
    # Assume `vectorstore` is your pre-defined vector store, converted to a retriever
    retriever = vectorstore.as_retriever()

    # Retrieve documents for two different queries
    retrieve_documents(prompt, retriever)

except Exception as e:
    print(f"An error occurred: {e}")

# Section 2
## Prompt enhancement with RAG and generating responses from Amazon Bedrock foundation model

#### 1. Prompt Creation (create_prompt):
###### Defines a system prompt using ChatPromptTemplate with guidance for concise, three-sentence responses.
###### Includes a {context} placeholder for contextual information and {input} for user queries.

#### 2. RAG Chain Creation (create_rag_chain):
###### Builds a RAG chain by combining a language model with retrieval.
###### Creates a question-answering sub-chain (question_answer_chain) and combines it with the retriever to form rag_chain.

#### 3. Main Execution Function (generate_responses_with_rag):
###### Initializes the language model (ChatBedrock) with a specific model ID and client.
###### Creates a prompt and sets up the retriever (assuming it's defined elsewhere).
###### Invokes the RAG chain with a sample question and handles exceptions to print any errors during execution.

In [ ]:
def create_prompt():
    """Create the system prompt for the ChatPromptTemplate."""
    system_prompt = (
        "You are an assistant designed for answering questions. "
        "Utilize the following context to provide your response. "
        "If you are unsure of the answer, please indicate that you "
        "do not know. Limit your response to a maximum of three sentences, "
        "ensuring that your answer is concise."
        "\n\n"
        "{context}"
    )
    return ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

def create_rag_chain(llm, prompt, retriever):
    """Create the Retrieval-Augmented Generation (RAG) chain."""
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    return rag_chain

def generate_responses_with_rag():
    """Main function to run the RAG workflow."""

    # Initialize the language model with the Bedrock client
    llm = ChatBedrock(model_id=bedrock_model_id,
                      client=boto3_bedrock_runtime_client)

    # Create the prompt for the question-answering task
    gen_prompt = create_prompt()

    # Initialize the retriever (assumed to be defined elsewhere)
    # retriever = AmazonKnowledgeBasesRetriever()  # Modify as needed

    # Create the RAG chain
    rag_chain = create_rag_chain(llm, gen_prompt, retriever)

    # Invoke the RAG chain with a sample question
    try:
        results = rag_chain.invoke({"input": prompt})
        print(results)
    except Exception as e:
        print(f"An error occurred during the RAG chain invocation: {e}")

generate_responses_with_rag()

# You shouod try to generate responses without RAG and compare the response with RAG result. 

# Generating responses from Amazon Bedrock foundation model without RAG

### 1. Language Model Initialization:
##### The llm is initialized with ChatBedrock, using bedrock_model_id and boto3_bedrock_runtime_client.

### 2. System Prompt Definition:
##### system_prompt is a concise instruction set for the assistant, enforcing brevity (three sentences max) and advising it to state if it doesn’t know the answer.

### 3. Response Generation Function (generate_response):
##### Combines system_prompt and the user’s question (question) to form a complete input.
##### Uses llm.invoke(input_text) to obtain a response from the Bedrock model.
##### Returns the model’s response content.

In [ ]:
# Initialize the language model
llm = ChatBedrock(
    model_id=bedrock_model_id,
    client=boto3_bedrock_runtime_client
)

# Define the system prompt
system_prompt = (
    "You are an assistant designed for answering questions. "
    "If you are unsure of the answer, please indicate that you "
    "do not know. Limit your response to a maximum of three sentences, "
    "ensuring that your answer is concise."
)

# Function to generate a response to a question
def generate_response(question):
    # Combine the system prompt and the user question
    input_text = f"{system_prompt}\n\n{question}"
    
    # Invoke the Bedrock model with the input text as a string
    response = llm.invoke(input_text)
    
    # Extract the content of the response
    return response

# Example usage
response = generate_response(prompt)

# Print the result
print(response)

# Must Read

######  Please have a look of the result of both the cases. Case 1 is generating more context aware information compare to Case 2.

###### Case1: Prompt enhancement with RAG and generating responses from Amazon Bedrock foundation model
###### Case2: Generating responses from Amazon Bedrock foundation model without RAG

# End of NoteBook 

## Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

## Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html